In [1]:
from filter import *
import pandas as pd
import os
import cv2
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import csv
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
%matplotlib inline
from libKMCUDA import kmeans_cuda
from scipy.cluster.vq import vq

In [2]:
# read data from disk to memory
def readData(path): 
    all_food_data = pd.read_csv(path, sep="\t", names=['imageName', 'dishName'])

    all_pic_names = all_food_data.drop('dishName', axis=1)
    all_dish_name = all_food_data['dishName']


    return all_pic_names, all_dish_name

In [3]:
# read data from disk to memory
def readData2(path): 
    all_food_data = pd.read_csv(path, sep="\t", names=['imageName', 'dishName'])

    x = all_food_data.drop('dishName', axis=1)
    y = all_food_data['dishName']

    #Dropped all multilabel data
    y=y.str.split(" ", n = 1, expand = True)
    data =  pd.concat([x, y], axis=1, join='inner')
    data.columns = ['imageName', 'dishName', 'trivial']
    data = data[pd.isnull(data['trivial'])]
    data = data.drop('trivial', axis=1)
    data = data.reset_index()
    del data['index']
    
    all_pic_names = data.drop('dishName', axis=1)
    all_dish_name = data['dishName']

    
    return all_pic_names, all_dish_name

In [4]:
def extractSchmidResponses(img_paths):
    schmid_filters = make_schmid_filters()
    responses = np.array([])
    base_path = '../Data/UNICT-FD1200_Small/'
    
    full_path = base_path + img_paths.iloc[0]['imageName']
    print(full_path)
    img = cv2.imread(full_path)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    responses =  [apply_filter_bank(lab, schmid_filters)]
    
    for path in img_paths.iloc[1:]['imageName']:
        full_path = base_path + path
        img = cv2.imread(full_path)
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        tmp = apply_filter_bank(lab, schmid_filters)
        responses = np.append(responses, [tmp] , axis=0)
        print(path)
    return responses

In [45]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

In [6]:
x_train_path, y_train = readData("Train_1.txt")
x_test_path, y_test = readData2("Test_1.txt")

In [46]:
savedResponsesFiles = os.listdir('files')
if 'train_responses12000Lab.npy' not in savedResponsesFiles:
    print('Train response features not found!')
    train_response_vector = extractSchmidResponses(x_train_path)
    np.save('files/train_responses12000Lab', train_response_vector)
    print('Train response features created!')
else:
    print('Train response fetaures found!')
    train_response_vector = np.load('files/train_responses12000Lab.npy')
    print('Train response features loaded!')

Train response fetaures found!
Train response features loaded!


In [48]:
train_response_vector.shape

(1200, 76800, 21)

In [49]:
train_response_vector = train_response_vector.reshape(train_response_vector.shape[0]*train_response_vector.shape[1], 21) # convert from (1200, 76800)
train_response_vector.shape

(92160000, 21)

In [9]:
def clusterResponses(responses):    
    #kmeans = KMeans(init='k-means++', n_clusters=1200, verbose=2, a)
    #kmeans = MiniBatchKMeans(init='k-means++', n_clusters=1200, batch_size=360000 ,verbose=2)
    responses = np.split(responses, 8)
    #print(str((responses[0].nbytes/1024)/1024))
    centroids, _ = kmeans_cuda(responses[0], 12002, init="k-means++",verbosity=2, yinyang_t=0)
    
    for arr in range(1,8):
        centroids, assignments = kmeans_cuda(arr, 12002, init=centroids, verbosity=2, yinyang_t=0)
    #kmeans.fit(responses)    
    #vocabulary = [kmeans.cluster_centers_]
    
    return centroids, assignments

In [17]:
#centroids, assigments = clusterResponses(train_response_vector)

ValueError: "samples" must be a 2D numpy array

In [7]:
centroids = np.load('files/centroids.npy')

In [19]:
np.argwhere(np.isnan(centroids))

array([[ 407,    0],
       [ 407,    1],
       [ 407,    2],
       [ 407,    3],
       [ 407,    4],
       [ 407,    5],
       [ 407,    6],
       [ 407,    7],
       [ 407,    8],
       [ 407,    9],
       [ 407,   10],
       [ 407,   11],
       [ 407,   12],
       [ 407,   13],
       [ 407,   14],
       [ 407,   15],
       [ 407,   16],
       [ 407,   17],
       [ 407,   18],
       [ 407,   19],
       [ 407,   20],
       [6657,    0],
       [6657,    1],
       [6657,    2],
       [6657,    3],
       [6657,    4],
       [6657,    5],
       [6657,    6],
       [6657,    7],
       [6657,    8],
       [6657,    9],
       [6657,   10],
       [6657,   11],
       [6657,   12],
       [6657,   13],
       [6657,   14],
       [6657,   15],
       [6657,   16],
       [6657,   17],
       [6657,   18],
       [6657,   19],
       [6657,   20]])

In [51]:
kmeans = KMeans(init=centroids, n_clusters=12000, verbose=2, max_iter=1)
kmeans.fit(train_response_vector[:12000])

/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:971: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


Initialization complete
start iteration
done sorting
end inner loop
Iteration 0, inertia 4529.829


/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (9165) found smaller than n_clusters (12000). Possibly due to duplicate points in X.
  return_n_iter=True)


KMeans(algorithm='auto', copy_x=True,
    init=array([[ 0.27989,  0.56903, ..., -0.03399, -0.01511],
       [ 0.28378,  0.51735, ..., -0.1438 ,  0.22932],
       ...,
       [ 0.4125 ,  0.50819, ..., -0.40991, -0.10592],
       [ 0.46805,  0.49457, ..., -0.72385, -0.47815]], dtype=float32),
    max_iter=1, n_clusters=12000, n_init=10, n_jobs=None,
    precompute_distances='auto', random_state=None, tol=0.0001, verbose=2)

In [8]:
centroids = np.array(np.delete(centroids, 407, 0))
centroids = np.array(np.delete(centroids, 6656, 0))
np.any(np.isnan(centroids))

False

In [23]:
np.save('vocabulary200NoMultiLabel', vocabulary)

In [11]:
vocabulary = np.array(vocabulary)
vocabulary = vocabulary.reshape(vocabulary.shape[0]*vocabulary.shape[1], 21)

In [39]:
test = extractSchmidResponses(x_test_path[1511:1512])

../Data/UNICT-FD1200_Small/_Sample_0622_0005.jpg


In [40]:
test = np.array(test).reshape(76800, 21)
test.shape

(76800, 21)

In [41]:
#test = np.array(test)
#test = test.reshape(76800,21)

In [53]:
hist = build_histogram(test, kmeans)

In [58]:
print(*hist)

0.0 0.0 0.0 1.0 6.0 0.0 0.0 1.0 3.0 0.0 0.0 3.0 0.0 0.0 22.0 5.0 1.0 0.0 5.0 13.0 22.0 0.0 17.0 2.0 0.0 0.0 0.0 0.0 17.0 2.0 2.0 0.0 3.0 2.0 1.0 3.0 0.0 7.0 6.0 1.0 0.0 1.0 5.0 0.0 0.0 0.0 2.0 1.0 3.0 1.0 15.0 0.0 0.0 6.0 7.0 81.0 27.0 2.0 3.0 0.0 0.0 2.0 0.0 2.0 2.0 3.0 1.0 4.0 0.0 2.0 10.0 0.0 13.0 6.0 2.0 0.0 0.0 5.0 1.0 0.0 1.0 1.0 14.0 0.0 9.0 5.0 3.0 3.0 6.0 1.0 2.0 2.0 4.0 16.0 5.0 12.0 0.0 5.0 6.0 10.0 3.0 1.0 20.0 10.0 2.0 7.0 4.0 3.0 27.0 3.0 0.0 3.0 8.0 0.0 4.0 3.0 1.0 10.0 5.0 0.0 0.0 1.0 2.0 2.0 6.0 0.0 5.0 0.0 1.0 22.0 4.0 7.0 0.0 4.0 2.0 35.0 4.0 5.0 1.0 3.0 5.0 8.0 0.0 0.0 6.0 6.0 0.0 1.0 1.0 2.0 9.0 1.0 2.0 3.0 2.0 10.0 3.0 29.0 2.0 2.0 7.0 0.0 2.0 4.0 0.0 6.0 1.0 4.0 10.0 1.0 8.0 4.0 3.0 0.0 3.0 3.0 17.0 7.0 18.0 1.0 3.0 1.0 0.0 2.0 1.0 8.0 6.0 2.0 4.0 1.0 7.0 3.0 13.0 0.0 6.0 9.0 1.0 5.0 7.0 3.0 0.0 14.0 0.0 5.0 3.0 11.0 0.0 3.0 0.0 15.0 7.0 3.0 0.0 2.0 0.0 4.0 2.0 7.0 3.0 5.0 4.0 4.0 5.0 19.0 36.0 2.0 1.0 25.0 2.0 0.0 14.0 51.0 6.0 1.0 7.0 0.0 7.0 10.0 7.0 3.0 7.0 2

 13.0 3.0 5.0 8.0 2.0 15.0 6.0 2.0 27.0 10.0 11.0 4.0 12.0 7.0 5.0 11.0 1.0 8.0 3.0 6.0 8.0 9.0 5.0 10.0 5.0 7.0 2.0 3.0 2.0 22.0 5.0 9.0 20.0 230.0 1.0 2.0 6.0 17.0 0.0 4.0 31.0 0.0 1.0 16.0 27.0 2.0 7.0 7.0 8.0 5.0 6.0 6.0 3.0 4.0 3.0 8.0 4.0 7.0 1.0 4.0 8.0 1.0 4.0 50.0 1.0 0.0 24.0 6.0 4.0 30.0 3.0 12.0 8.0 1.0 9.0 7.0 17.0 0.0 8.0 2.0 3.0 5.0 4.0 12.0 4.0 2.0 10.0 7.0 3.0 10.0 0.0 7.0 3.0 1.0 6.0 0.0 3.0 9.0 6.0 7.0 5.0 6.0 0.0 1.0 4.0 2.0 2.0 4.0 10.0 7.0 18.0 4.0 3.0 7.0 37.0 0.0 4.0 3.0 1.0 1.0 6.0 4.0 4.0 14.0 0.0 6.0 4.0 2.0 8.0 5.0 3.0 2.0 8.0 8.0 3.0 21.0 26.0 0.0 0.0 3.0 4.0 3.0 23.0 5.0 7.0 0.0 0.0 11.0 0.0 3.0 17.0 5.0 17.0 0.0 2.0 7.0 2.0 2.0 8.0 12.0 4.0 4.0 15.0 8.0 9.0 7.0 4.0 2.0 0.0 8.0 6.0 2.0 0.0 0.0 31.0 1.0 13.0 24.0 0.0 15.0 7.0 1.0 7.0 26.0 10.0 0.0 0.0 4.0 19.0 57.0 15.0 1.0 5.0 8.0 1.0 8.0 3.0 3.0 8.0 15.0 2.0 3.0 12.0 2.0 0.0 9.0 4.0 11.0 0.0 6.0 1.0 1.0 18.0 2.0 13.0 28.0 0.0 13.0 2.0 11.0 2.0 2.0 1.0 1.0 0.0 7.0 11.0 5.0 0.0 9.0 46.0 15.0 7.0 8.0 7.0 5.0

1.0 6.0 27.0 10.0 0.0 7.0 2.0 9.0 38.0 24.0 1.0 9.0 4.0 3.0 4.0 0.0 2.0 2.0 3.0 0.0 8.0 8.0 4.0 6.0 0.0 0.0 4.0 3.0 10.0 4.0 12.0 3.0 2.0 3.0 0.0 21.0 17.0 1.0 5.0 34.0 4.0 9.0 2.0 0.0 0.0 1.0 37.0 2.0 53.0 16.0 3.0 1.0 9.0 4.0 2.0 0.0 0.0 1.0 4.0 0.0 25.0 149.0 4.0 15.0 4.0 4.0 34.0 8.0 9.0 2.0 0.0 1.0 2.0 2.0 9.0 2.0 2.0 2.0 14.0 0.0 0.0 21.0 2.0 0.0 2.0 21.0 1.0 19.0 0.0 2.0 0.0 6.0 1.0 5.0 3.0 1.0 3.0 0.0 7.0 5.0 3.0 0.0 4.0 10.0 4.0 1.0 6.0 0.0 4.0 8.0 6.0 8.0 4.0 3.0 0.0 0.0 0.0 5.0 0.0 1.0 1.0 8.0 5.0 4.0 0.0 0.0 1.0 4.0 30.0 3.0 13.0 2.0 106.0 4.0 4.0 4.0 4.0 0.0 2.0 3.0 3.0 4.0 0.0 3.0 2.0 2.0 0.0 2.0 13.0 1.0 0.0 3.0 4.0 15.0 3.0 1.0 4.0 7.0 2.0 17.0 5.0 27.0 0.0 0.0 1.0 8.0 3.0 10.0 13.0 0.0 8.0 0.0 3.0 5.0 2.0 9.0 12.0 41.0 0.0 14.0 10.0 3.0 19.0 2.0 5.0 0.0 8.0 6.0 0.0 4.0 7.0 8.0 9.0 11.0 0.0 1.0 1.0 4.0 17.0 3.0 14.0 4.0 2.0 5.0 2.0 0.0 6.0 0.0 0.0 19.0 0.0 1.0 2.0 9.0 2.0 15.0 7.0 3.0 14.0 7.0 5.0 2.0 1.0 16.0 4.0 10.0 8.0 4.0 1.0 2.0 0.0 0.0 6.0 11.0 5.0 0.0 2.0 1.0 5.

 7.0 2.0 5.0 1.0 3.0 2.0 4.0 6.0 0.0 4.0 28.0 11.0 4.0 12.0 16.0 0.0 1.0 0.0 0.0 5.0 1.0 8.0 0.0 0.0 1.0 0.0 5.0 0.0 4.0 0.0 1.0 1.0 10.0 2.0 4.0 3.0 25.0 3.0 0.0 0.0 1.0 0.0 0.0 53.0 0.0 8.0 0.0 1.0 8.0 4.0 7.0 0.0 0.0 6.0 4.0 5.0 12.0 0.0 8.0 0.0 1.0 0.0 1.0 0.0 12.0 19.0 24.0 0.0 4.0 51.0 9.0 1.0 0.0 0.0 0.0 0.0 3.0 3.0 0.0 9.0 2.0 7.0 4.0 3.0 10.0 11.0 2.0 2.0 0.0 4.0 0.0 3.0 0.0 8.0 12.0 14.0 7.0 0.0 3.0 19.0 1.0 9.0 0.0 1.0 1.0 0.0 1.0 1.0 14.0 0.0 3.0 4.0 0.0 8.0 4.0 20.0 15.0 10.0 7.0 13.0 4.0 12.0 3.0 9.0 11.0 0.0 0.0 0.0 0.0 5.0 4.0 5.0 2.0 4.0 1.0 10.0 20.0 11.0 0.0 15.0 2.0 1.0 10.0 0.0 19.0 8.0 5.0 0.0 2.0 19.0 15.0 1.0 11.0 12.0 5.0 2.0 0.0 3.0 8.0 8.0 3.0 9.0 2.0 14.0 10.0 9.0 0.0 27.0 4.0 4.0 1.0 1.0 6.0 6.0 2.0 6.0 3.0 4.0 0.0 14.0 17.0 0.0 0.0 3.0 23.0 16.0 1.0 16.0 0.0 7.0 5.0 7.0 0.0 1.0 3.0 0.0 4.0 4.0 0.0 11.0 4.0 7.0 2.0 5.0 18.0 0.0 2.0 3.0 8.0 2.0 0.0 5.0 0.0 6.0 0.0 8.0 22.0 0.0 4.0 11.0 3.0 5.0 14.0 26.0 0.0 4.0 3.0 0.0 6.0 52.0 8.0 8.0 5.0 16.0 2.0 5.0 5.0 0

In [15]:
preprocessed_image = []
for i in range(200):
    img = x_train_path.iloc[i:i+1]
    resp = extractSchmidResponses(img)
    resp = np.array(resp)
    resp = resp.reshape(resp.shape[0]*resp.shape[1],21)
    if (resp is not None):
        histogram = build_histogram(resp, model)
        preprocessed_image.append(histogram)

../Data/UNICT-FD1200_Small/_Sample_0001_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0002_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0003_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0004_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0005_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0007_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0008_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0009_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0010_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0011_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0012_0001.jpg
Computing l

Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0097_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0099_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0100_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0101_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0103_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0104_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0105_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0106_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0107_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0108_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-F

../Data/UNICT-FD1200_Small/_Sample_0204_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0205_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0206_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0207_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0208_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0209_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0210_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0211_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0212_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0214_0001.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0215_0001.jpg
Computing l

In [16]:
preprocessed_test_image = []
for i in range(200):
    img = x_test_path.iloc[i:i+1]
    resp = extractSchmidResponses(img)
    resp = np.array(resp)
    resp = resp.reshape(resp.shape[0]*resp.shape[1],21)
    if (resp is not None):
        histogram = build_histogram(resp, model)
        preprocessed_test_image.append(histogram)

../Data/UNICT-FD1200_Small/_Sample_0009_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0009_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0009_0004.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0009_0005.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0012_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0012_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0012_0004.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0014_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0014_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0015_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0015_0003.jpg
Computing l

Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0148_0004.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0148_0005.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0148_0006.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0148_0007.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0149_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0149_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0150_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0150_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0151_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0151_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-F

../Data/UNICT-FD1200_Small/_Sample_0191_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0191_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0192_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0192_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0193_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0193_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0194_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0194_0003.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0194_0004.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0195_0002.jpg
Computing label assignment and total inertia
../Data/UNICT-FD1200_Small/_Sample_0195_0003.jpg
Computing l

In [24]:
np.save('preprocessed_train_image_NL_200',preprocessed_image)
np.save('preprocessed_test_image_NL_200',preprocessed_test_image)

In [12]:
a = np.load('preprocessed_train_image_NL_200.npy')#np.array(preprocessed_image)

In [26]:
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn.fit(a, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [27]:
b = np.load('preprocessed_test_image_NL_200.npy')#preprocessed_test_image)

In [28]:
pr = knn.predict(b)

In [29]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(pr, y_test))

Accuracy: 0.575


In [30]:
from sklearn.metrics import classification_report, confusion_matrix  
print(classification_report(pr, y_test))  

               precision    recall  f1-score   support

    Appetizer       0.52      0.61      0.56        23
      Dessert       0.43      0.75      0.55         8
        Fruit       0.79      0.79      0.79        19
  Main_Course       0.62      0.58      0.60        52
Second_Course       0.63      0.52      0.57        61
    Side_Dish       0.58      0.50      0.54        30
Single_Course       0.20      0.43      0.27         7

    micro avg       0.57      0.57      0.57       200
    macro avg       0.54      0.60      0.55       200
 weighted avg       0.60      0.57      0.58       200



In [36]:
from pandas_ml import ConfusionMatrix
confusion_matrix = ConfusionMatrix(y_test, pr )
confusion_matrix.plt()
plt.show

AttributeError: 'BinaryConfusionMatrix' object has no attribute 'plt'

In [37]:
plot_confusion_matrix(y_test, pr, classes=y_test,
                      title='Confusion matrix, without normalization')

NameError: name 'plot_confusion_matrix' is not defined

In [ ]:
train_response_vector.shape

In [ ]:
train_response_vector = extractSchmidResponses(x_train_path)

In [ ]:
np.save('files/train_responses1200', train_response_vector)

In [ ]:
tc = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 0.001) # I don't know what the parameters mean
BoW = cv2.BOWKMeansTrainer(1200, tc,1,cv2.KMEANS_PP_CENTERS)# create a BagOfFeaturesWord. It can be take a long time 
vocabulary = BoW.cluster(train_response_vector)

In [ ]:
all_train_responses = extractSchmidResponses(x_path)

In [ ]:
np.save('files/train_responses.csv', all_train_responses)

In [ ]:
words = clusterResponses(all_train_responses)kme

In [ ]:
np.save('files/train_clusters.csv', all_train_responses)

In [ ]:
def calculate_centroids_histogram(voc, model):

    feature_vectors=[]
    class_vectors=[]
    
    for item in voc:
        predict_kmeans = model.predict(item)
        hist, bin_edges = np.histogram(predict_kmeans, 1200)
        feature_vectors.append(hist)
        
    feature_vectors = np.asarray(feature_vectors)
    
    return feature_vectors

In [ ]:
kmeans = KMeans(n_clusters=1200).fit(new_words)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(words, y, test_size=0.25)

In [ ]:
train = calculate_centroids_histogram(X_train, kmeans)

In [ ]:
test = calculate_centroids_histogram(X_test, kmeans)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_image, y, test_size=0.25)

In [ ]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(train)

train = scaler.transform(train)  
test = scaler.transform(test)  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=10)  
classifier.fit(train, y_train) 

In [ ]:
y_pred = classifier.predict(test)  

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))  

In [ ]:
#from sklearn.svm import SVC  
#svclassifier = SVC(kernel='poly', degree=8)  
#svclassifier.fit(preprocessed_image, y_train)  

In [ ]:
y_pred = svclassifier.predict(b)  

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))  
